# imports

In [1]:
# essential imports
import pandas as pd 
import numpy as np 

In [15]:
# model imports
from xgboost import XGBClassifier as xgb
from sklearn.model_selection import GridSearchCV as gs 

# model tools 
from sklearn.model_selection import train_test_split

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

from sklearn.preprocessing import StandardScaler
 

# data import

In [58]:
# Read data
df1 = pd.read_csv('match_model.csv',index_col= 0)

In [59]:
# drop unwanted cols
df = df1.drop(columns= ['Season','Round'])

In [60]:
strip = lambda x : x.strip(' ')
df.main = df['main'].apply(strip)
df.opp = df['opp'].apply(strip)
df.winner = df['winner'].apply(strip)

# Classification

In [94]:
# features
X = pd.DataFrame(df[['main','opp']])

# Select dependent variable
y = pd.DataFrame(df[['winner']])

In [95]:
X

,main,opp
0,Switchback,Banshee
1,Banshee,Valkyrie
2,Emulsifier,Banshee
4,Kraken,Beta
5,Beta,Shatter
...,...,...
4950,Lock-Jaw,Yeti
4951,Chomp,Yeti
4952,Yeti,Lucky
4953,Tombstone,Yeti


In [96]:
y

,winner
0,Switchback
1,Banshee
2,Emulsifier
4,Beta
5,Beta
...,...
4950,Yeti
4951,Yeti
4952,Yeti
4953,Tombstone


# Encoding Categorical Data

In [97]:
print(X)

            main       opp
0     Switchback   Banshee
1        Banshee  Valkyrie
2     Emulsifier   Banshee
4         Kraken      Beta
5           Beta   Shatter
...          ...       ...
4950    Lock-Jaw      Yeti
4951       Chomp      Yeti
4952        Yeti     Lucky
4953   Tombstone      Yeti
4954  Mega Tento      Yeti

[4842 rows x 2 columns]


In [98]:
X = pd.get_dummies(df[['main','opp']])

In [103]:
# Convert the target variable to numeric values using one-hot encoding
y = pd.get_dummies(df['winner'])


# splitting data

In [104]:
# split the data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

# scaling

# classifcation

In [108]:
from sklearn.model_selection import GridSearchCV, train_test_split
import xgboost as xgb

# Define the XGBoost model
xgb_model = xgb.XGBClassifier()

# Define the hyperparameters and their potential values to be tuned
parameters = {'max_depth': [3, 4, 5], 'learning_rate': [0.1, 0.01, 0.001], 'n_estimators': [100, 500, 1000]}

# Perform grid search using cross-validation
grid_search = GridSearchCV(xgb_model, parameters, scoring='accuracy', n_jobs=-1, cv=5)

# Fit the grid search object to the training data
grid_search.fit(X_train, y_train, xgb_model__eval_metric='logloss')

# Print the best hyperparameters and their score
print('Best hyperparameters: ', grid_search.best_params_)
print('Best score: ', grid_search.best_score_)

# Use the best model to make predictions on the test set
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)



ValueError: 
All the 135 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
135 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/anaconda3/lib/python3.9/site-packages/xgboost/core.py", line 620, in inner_f
    return func(**kwargs)
TypeError: fit() got an unexpected keyword argument 'xgb_model__early_stopping_rounds'


In [112]:
import xgboost as xgb
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score

params = {
    'max_depth': [3, 5],
    'learning_rate': [0.1, 0.01],
    'subsample': [0.8, 0.9],
    'colsample_bytree': [0.8, 0.9],
    'n_estimators': [50, 100]
}

model = xgb.XGBClassifier(objective='binary:logistic', seed=42)

grid_search = GridSearchCV(estimator=model, param_grid=params, cv=3, verbose=3)
grid_search.fit(X_train, y_train)

best_params = grid_search.best_params_
print("Best params:", best_params)

best_model = xgb.XGBClassifier(**best_params)
best_model.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_test, y_test)],
               early_stopping_rounds=10, verbose=10)

y_pred = best_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Test accuracy:", accuracy)



Fitting 3 folds for each of 32 candidates, totalling 96 fits
[CV 1/3] END colsample_bytree=0.8, learning_rate=0.1, max_depth=3, n_estimators=50, subsample=0.8;, score=0.564 total time= 1.1min
[CV 2/3] END colsample_bytree=0.8, learning_rate=0.1, max_depth=3, n_estimators=50, subsample=0.8;, score=0.618 total time=  55.4s
[CV 3/3] END colsample_bytree=0.8, learning_rate=0.1, max_depth=3, n_estimators=50, subsample=0.8;, score=0.569 total time=  50.0s
[CV 1/3] END colsample_bytree=0.8, learning_rate=0.1, max_depth=3, n_estimators=50, subsample=0.9;, score=0.576 total time= 1.1min
[CV 2/3] END colsample_bytree=0.8, learning_rate=0.1, max_depth=3, n_estimators=50, subsample=0.9;, score=0.632 total time= 1.1min
[CV 3/3] END colsample_bytree=0.8, learning_rate=0.1, max_depth=3, n_estimators=50, subsample=0.9;, score=0.580 total time= 1.1min
[CV 1/3] END colsample_bytree=0.8, learning_rate=0.1, max_depth=3, n_estimators=100, subsample=0.8;, score=0.624 total time= 1.8min
[CV 2/3] END colsampl

[CV 3/3] END colsample_bytree=0.9, learning_rate=0.1, max_depth=5, n_estimators=50, subsample=0.8;, score=0.593 total time=  50.2s
[CV 1/3] END colsample_bytree=0.9, learning_rate=0.1, max_depth=5, n_estimators=50, subsample=0.9;, score=0.607 total time=  53.7s
[CV 2/3] END colsample_bytree=0.9, learning_rate=0.1, max_depth=5, n_estimators=50, subsample=0.9;, score=0.637 total time=  54.6s
[CV 3/3] END colsample_bytree=0.9, learning_rate=0.1, max_depth=5, n_estimators=50, subsample=0.9;, score=0.590 total time=  55.6s
[CV 1/3] END colsample_bytree=0.9, learning_rate=0.1, max_depth=5, n_estimators=100, subsample=0.8;, score=0.636 total time= 1.6min
[CV 2/3] END colsample_bytree=0.9, learning_rate=0.1, max_depth=5, n_estimators=100, subsample=0.8;, score=0.674 total time= 1.5min
[CV 3/3] END colsample_bytree=0.9, learning_rate=0.1, max_depth=5, n_estimators=100, subsample=0.8;, score=0.636 total time= 1.5min
[CV 1/3] END colsample_bytree=0.9, learning_rate=0.1, max_depth=5, n_estimators=

/usr/local/anaconda3/lib/python3.9/site-packages/xgboost/sklearn.py:835: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


[0]	validation_0-logloss:0.59927	validation_1-logloss:0.59940
[10]	validation_0-logloss:0.18211	validation_1-logloss:0.18297
[20]	validation_0-logloss:0.06648	validation_1-logloss:0.06769
[30]	validation_0-logloss:0.02713	validation_1-logloss:0.02852
[40]	validation_0-logloss:0.01276	validation_1-logloss:0.01424
[50]	validation_0-logloss:0.00735	validation_1-logloss:0.00889
[60]	validation_0-logloss:0.00527	validation_1-logloss:0.00685
[70]	validation_0-logloss:0.00442	validation_1-logloss:0.00601
[80]	validation_0-logloss:0.00404	validation_1-logloss:0.00564
[90]	validation_0-logloss:0.00388	validation_1-logloss:0.00550
[99]	validation_0-logloss:0.00381	validation_1-logloss:0.00545
Test accuracy: 0.8369453044375645


In [113]:
best_model

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=5, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [117]:
best_model.get_params()

{'objective': 'binary:logistic',
 'use_label_encoder': None,
 'base_score': None,
 'booster': None,
 'callbacks': None,
 'colsample_bylevel': None,
 'colsample_bynode': None,
 'colsample_bytree': 0.8,
 'early_stopping_rounds': None,
 'enable_categorical': False,
 'eval_metric': None,
 'feature_types': None,
 'gamma': None,
 'gpu_id': None,
 'grow_policy': None,
 'importance_type': None,
 'interaction_constraints': None,
 'learning_rate': 0.1,
 'max_bin': None,
 'max_cat_threshold': None,
 'max_cat_to_onehot': None,
 'max_delta_step': None,
 'max_depth': 5,
 'max_leaves': None,
 'min_child_weight': None,
 'missing': nan,
 'monotone_constraints': None,
 'n_estimators': 100,
 'n_jobs': None,
 'num_parallel_tree': None,
 'predictor': None,
 'random_state': None,
 'reg_alpha': None,
 'reg_lambda': None,
 'sampling_method': None,
 'scale_pos_weight': None,
 'subsample': 0.9,
 'tree_method': None,
 'validate_parameters': None,
 'verbosity': None}

In [ ]:
'''{'objective': 'binary:logistic',
 'use_label_encoder': None,
 'base_score': None,
 'booster': None,
 'callbacks': None,
 'colsample_bylevel': None,
 'colsample_bynode': None,
 'colsample_bytree': 0.8,
 'early_stopping_rounds': None,
 'enable_categorical': False,
 'eval_metric': None,
 'feature_types': None,
 'gamma': None,
 'gpu_id': None,
 'grow_policy': None,
 'importance_type': None,
 'interaction_constraints': None,
 'learning_rate': 0.1,
 'max_bin': None,
 'max_cat_threshold': None,
 'max_cat_to_onehot': None,
 'max_delta_step': None,
 'max_depth': 5,
 'max_leaves': None,
 'min_child_weight': None,
 'missing': nan,
 'monotone_constraints': None,
 'n_estimators': 100,
 'n_jobs': None,
 'num_parallel_tree': None,
 'predictor': None,
 'random_state': None,
 'reg_alpha': None,
 'reg_lambda': None,
 'sampling_method': None,
 'scale_pos_weight': None,
 'subsample': 0.9,
 'tree_method': None,
 'validate_parameters': None,
 'verbosity': None}'''